#### In this notebook, we will use the [U.S. BUREAU OF LABOR STATISTICS's](https://www.bls.gov/emp/tables/emp-by-detailed-occupation.htm) employment data to map 'occupations' (first employment category), with the second level of employment categories, and eventually their respective employment titles (the final level of the 3-part employment hierarchy).  
The final output, *Cleaned_BLS_Employment_Categories.csv*, should be a normalized table with all employment titles mapped to their second and first level occupation categories. 

In [1]:
# Importing dependencies
import pandas as pd
import numpy as np

In [2]:
# Read Data
dataset = pd.read_excel('Resources/occupation.xlsx', sheet_name='Table 1.2', skiprows = range(1),
                   usecols = "A:C")
columns = ["Employment_Title", "Employment_Code", "Employment_Type"]
dataset.columns = columns                  
print(dataset.head(10))

                                    Employment_Title Employment_Code  \
0                                                NaN             NaN   
1                                                NaN             NaN   
2                             Total, all occupations         00-0000   
3                             Management occupations         11-0000   
4                                     Top executives         11-1000   
5                                   Chief executives         11-1011   
6                    General and operations managers         11-1021   
7                                        Legislators         11-1031   
8  Advertising, marketing, promotions, public rel...         11-2000   
9                Advertising and promotions managers         11-2011   

  Employment_Type  
0             NaN  
1             NaN  
2         Summary  
3         Summary  
4         Summary  
5       Line item  
6       Line item  
7       Line item  
8         Summary  
9      

In [3]:
# Drop all rows where is NULL
dataset.dropna(axis=0, how='any', inplace = True) #Visual inspection shows we can drop 'any'
dataset = dataset.reset_index(drop=True)
dataset

,Employment_Title,Employment_Code,Employment_Type
0,"Total, all occupations",00-0000,Summary
1,Management occupations,11-0000,Summary
2,Top executives,11-1000,Summary
3,Chief executives,11-1011,Line item
4,General and operations managers,11-1021,Line item
...,...,...,...
1043,"Pump operators, except wellhead pumpers",53-7072,Line item
1044,Wellhead pumpers,53-7073,Line item
1045,Refuse and recyclable material collectors,53-7081,Line item
1046,"Tank car, truck, and ship loaders",53-7121,Line item


In [4]:
#Checking
dataset.isnull().sum()
# dataset.isnull()

Employment_Title    0
Employment_Code     0
Employment_Type     0
dtype: int64

In [5]:
# Exclude 1st row
dataset = dataset.iloc[1:len(dataset)+1]
dataset = dataset.reset_index(drop=True)
dataset

,Employment_Title,Employment_Code,Employment_Type
0,Management occupations,11-0000,Summary
1,Top executives,11-1000,Summary
2,Chief executives,11-1011,Line item
3,General and operations managers,11-1021,Line item
4,Legislators,11-1031,Line item
...,...,...,...
1042,"Pump operators, except wellhead pumpers",53-7072,Line item
1043,Wellhead pumpers,53-7073,Line item
1044,Refuse and recyclable material collectors,53-7081,Line item
1045,"Tank car, truck, and ship loaders",53-7121,Line item


In [6]:
# Split Employment_Code column
def split_columns(df, split_column, on_char, new_col_1, new_col_2, drop=True):
    print((on_char))
    new = df[split_column].str.split(on_char, n = 1, expand = True) 
    
    # making separate left-side column using new dataframe 
    df[new_col_1]= new[0]
    
    # making separate right-side column using new dataframe 
    df[new_col_2]= new[1]

    # Dropping old Name columns
    if drop == True:
        df.drop(columns =[split_column], inplace = True)

    return df
  
# return df
dataset = split_columns(dataset, "Employment_Code", "-", "Employment_Code_0", "Employment_Code_1", drop=False)
dataset 

-


,Employment_Title,Employment_Code,Employment_Type,Employment_Code_0,Employment_Code_1
0,Management occupations,11-0000,Summary,11,0000
1,Top executives,11-1000,Summary,11,1000
2,Chief executives,11-1011,Line item,11,1011
3,General and operations managers,11-1021,Line item,11,1021
4,Legislators,11-1031,Line item,11,1031
...,...,...,...,...,...
1042,"Pump operators, except wellhead pumpers",53-7072,Line item,53,7072
1043,Wellhead pumpers,53-7073,Line item,53,7073
1044,Refuse and recyclable material collectors,53-7081,Line item,53,7081
1045,"Tank car, truck, and ship loaders",53-7121,Line item,53,7121


In [7]:
# Get an array of unique (left-most) empl. codes
employment_code_array = dataset['Employment_Code_0'].to_numpy()
employment_code_array = np.unique(employment_code_array) #get unique names
employment_code_array.shape

(22,)

In [8]:
employment_level_1_arr = []
employment_level_2_arr = []
employment_title_arr = []

# Feel free to wrap this into a function
# At a high-level: we will map the unique codes from the employment_code_array with the first, second, and third level employment titles in the dataset
# This for loop creates arrays of with key-value pairs that we will use to perform mapping / join operations later

# Loop through unique set of employment codes
for i in range(len(employment_code_array)):
    # Within that, loop through the whole dataset
    for j in range(len(dataset)):
        # If the unique code matches that row's Employment_Code_0 value
        if dataset.loc[j, 'Employment_Code_0'] == employment_code_array[i]:
            # And if it ends in '0000', then it's a first level of employment category (first level 'summary') 
            if dataset.loc[j, 'Employment_Code_1'] == '0000':
                # Map the BLS key of that 1st level summary with its corresponding name
                temp_key = f"{dataset.loc[j, 'Employment_Code_0']}-0000"
                employment_level_1_arr.append({temp_key: dataset.loc[j, 'Employment_Title']})
            # If the row doesn't meet our '0000' condition
            # But it ends with '000' then it's the 2nd level summary
            # There are additional levels for some 2nd level summaries but the 2nd level seems like a sufficient place to end
            elif str(dataset.loc[j, 'Employment_Code_1']).endswith("000"):
                # Create a temp key that combines the current 1st level 'temp_key' with the 2nd level's BLS key
                new_temp_key = f"{dataset.loc[j, 'Employment_Code_0']}-{dataset.loc[j, 'Employment_Code_1']}"
                employment_level_2_arr.append({f"{temp_key}+{new_temp_key}": dataset.loc[j, 'Employment_Title']})
            else: 
                # Else this is an employment title (third and 'last' level)
                # Map the current 'new_temp_key' with it so we can perform our joins later.
                employment_title_arr.append({new_temp_key: dataset.loc[j, 'Employment_Title']})

# We'll get create our objects within the arrays we've declared earlier
# These objects will result in key-value pairs
# Code idea source: https://bit.ly/3rooILz by 'Nayanexx.py'
employment_level_1_arr = [{
    k: [d.get(k) for d in employment_level_1_arr if k in d]
    for k in set().union(*employment_level_1_arr)
  }]

employment_level_2_arr = [{
    k: [d.get(k) for d in employment_level_2_arr if k in d]
    for k in set().union(*employment_level_2_arr)
  }]

employment_title_arr = [{
    k: [d.get(k) for d in employment_title_arr if k in d]
    for k in set().union(*employment_title_arr)
  }]

In [9]:
employment_level_1_arr

[{'47-0000': ['Construction and extraction occupations'],
  '17-0000': ['Architecture and engineering occupations'],
  '31-0000': ['Healthcare support occupations'],
  '27-0000': ['Arts, design, entertainment, sports, and media occupations'],
  '11-0000': ['Management occupations'],
  '43-0000': ['Office and administrative support occupations'],
  '33-0000': ['Protective service occupations'],
  '21-0000': ['Community and social service occupations'],
  '25-0000': ['Educational instruction and library occupations'],
  '13-0000': ['Business and financial operations occupations'],
  '23-0000': ['Legal occupations'],
  '41-0000': ['Sales and related occupations'],
  '19-0000': ['Life, physical, and social science occupations'],
  '53-0000': ['Transportation and material moving occupations'],
  '15-0000': ['Computer and mathematical occupations'],
  '37-0000': ['Building and grounds cleaning and maintenance occupations'],
  '35-0000': ['Food preparation and serving related occupations'],
 

In [10]:
employment_level_2_arr

[{'37-0000+37-3000': ['Grounds maintenance workers'],
  '53-0000+53-5000': ['Water transportation workers'],
  '25-0000+25-3000': ['Other teachers and instructors'],
  '25-0000+25-4000': ['Librarians, curators, and archivists'],
  '49-0000+49-2000': ['Electrical and electronic equipment mechanics, installers, and repairers'],
  '47-0000+47-4000': ['Other construction and related workers'],
  '23-0000+23-2000': ['Legal support workers'],
  '13-0000+13-2000': ['Financial specialists'],
  '47-0000+47-5000': ['Extraction workers'],
  '51-0000+51-9000': ['Other production occupations'],
  '51-0000+51-4000': ['Metal workers and plastic workers'],
  '51-0000+51-8000': ['Plant and system operators'],
  '51-0000+51-3000': ['Food processing workers'],
  '31-0000+31-2000': ['Occupational therapy and physical therapist assistants and aides'],
  '33-0000+33-2000': ['Firefighting and prevention workers'],
  '39-0000+39-6000': ['Baggage porters, bellhops, and concierges'],
  '35-0000+35-2000': ['Cook

In [11]:
# employment_title_arr

In [12]:
# Let's push these into pandas df
# Function to return keys
def getList(dict): 
    list = [] 
    for key in dict.keys(): 
        list.append(key) 
        
    return list

# Function to return df
def return_df(arr, columns):
    df = pd.DataFrame(columns=columns)
    key_column = columns[0]
    name_column = columns[1]
    keys_array = getList(arr[0])
    # Incrementally fill the df
    for i, k in enumerate(keys_array):
        df.loc[i, key_column] = k
        if len(df.filter(like='Employment_Title').columns) == 0: #We need the array of titles here
            df.loc[i, name_column] = arr[0].get(k)[0]
        else:
            df.loc[i, name_column] = arr[0].get(k)
        
    return df

#Return dfs
columns = ['Primary_Level_Key', 'Employment_Level_1_Title']
employment_level_1_df = return_df(employment_level_1_arr, columns)
columns = ['Primary_Level_Key_0', 'Employment_Level_2_Title']
employment_level_2_df = return_df(employment_level_2_arr, columns)
columns = ['Secondary_Level_Key', 'Employment_Title']
employment_title_df = return_df(employment_title_arr, columns)
employment_title_df.head(10)

,Secondary_Level_Key,Employment_Title
0,49-1000,"[First-line supervisors of mechanics, installe..."
1,25-9000,"[Farm and home management educators, Instructi..."
2,21-1000,"[Counselors, Educational, guidance, and career..."
3,53-5000,"[Sailors and marine oilers, Ship and boat capt..."
4,45-2000,"[Agricultural inspectors, Animal breeders, Gra..."
5,19-4000,"[Agricultural and food science technicians, Bi..."
6,29-9000,"[Athletic trainers, Genetic counselors, Health..."
7,27-3000,"[Broadcast announcers and radio disc jockeys, ..."
8,13-2000,"[Accountants and auditors, Property appraisers..."
9,51-7000,"[Cabinetmakers and bench carpenters, Furniture..."


In [13]:
# Split Employment_Level_2 column
employment_level_2_df = split_columns(employment_level_2_df, "Primary_Level_Key_0", "+", "Primary_Level_Key", "Secondary_Level_Key")
employment_level_2_df 

+


,Employment_Level_2_Title,Primary_Level_Key,Secondary_Level_Key
0,Grounds maintenance workers,37-0000,37-3000
1,Water transportation workers,53-0000,53-5000
2,Other teachers and instructors,25-0000,25-3000
3,"Librarians, curators, and archivists",25-0000,25-4000
4,"Electrical and electronic equipment mechanics,...",49-0000,49-2000
...,...,...,...
85,Financial clerks,43-0000,43-3000
86,Material moving workers,53-0000,53-7000
87,Supervisors of office and administrative suppo...,43-0000,43-1000
88,Health technologists and technicians,29-0000,29-2000


In [14]:
# Will we expand or 'explode' our arrays in 'Employment_Title' in the Employment_Title df with their corresponding temp_keys we set
employment_title_df = employment_title_df.explode("Employment_Title")
employment_title_df = employment_title_df.reset_index(drop=True)
employment_title_df

,Secondary_Level_Key,Employment_Title
0,49-1000,"First-line supervisors of mechanics, installer..."
1,25-9000,Farm and home management educators
2,25-9000,Instructional coordinators
3,25-9000,Teaching assistants
4,25-9000,"Teaching assistants, postsecondary"
...,...,...
930,25-3000,Self-enrichment teachers
931,25-3000,"Substitute teachers, short-term"
932,25-3000,"Tutors and teachers and instructors, all other"
933,41-1000,First-line supervisors of retail sales workers


In [15]:
# Now let's join our dfs by key
from functools import reduce

final_output_df = reduce(lambda left, right: pd.merge(left, right, how='outer', on=['Primary_Level_Key']), [employment_level_1_df, employment_level_2_df])
final_output_df = reduce(lambda left, right: pd.merge(left, right, how='outer', on=['Secondary_Level_Key']), [final_output_df, employment_title_df])

# Also merge with original keys from BLS file - note the join on 'left' to exclude second_level designations since that info is captured
final_output_df = reduce(lambda left, right: pd.merge(left, right, how='left', on=['Employment_Title']), [final_output_df, dataset])
final_output_df = final_output_df.drop_duplicates()

final_output_df.describe()

,Primary_Level_Key,Employment_Level_1_Title,Employment_Level_2_Title,Secondary_Level_Key,Employment_Title,Employment_Code,Employment_Type,Employment_Code_0,Employment_Code_1
count,937,937,937,937,937,937,937,937,937
unique,22,22,90,90,934,937,2,22,428
top,51-0000,Production occupations,Postsecondary teachers,25-1000,Tour and travel guides,25-1020,Line item,51,2011
freq,131,131,47,47,2,1,790,131,21


In [16]:
final_output_df.head(10)

,Primary_Level_Key,Employment_Level_1_Title,Employment_Level_2_Title,Secondary_Level_Key,Employment_Title,Employment_Code,Employment_Type,Employment_Code_0,Employment_Code_1
0,47-0000,Construction and extraction occupations,Other construction and related workers,47-4000,Construction and building inspectors,47-4011,Line item,47,4011
1,47-0000,Construction and extraction occupations,Other construction and related workers,47-4000,Elevator and escalator installers and repairers,47-4021,Line item,47,4021
2,47-0000,Construction and extraction occupations,Other construction and related workers,47-4000,Fence erectors,47-4031,Line item,47,4031
3,47-0000,Construction and extraction occupations,Other construction and related workers,47-4000,Hazardous materials removal workers,47-4041,Line item,47,4041
4,47-0000,Construction and extraction occupations,Other construction and related workers,47-4000,Highway maintenance workers,47-4051,Line item,47,4051
5,47-0000,Construction and extraction occupations,Other construction and related workers,47-4000,Rail-track laying and maintenance equipment op...,47-4061,Line item,47,4061
6,47-0000,Construction and extraction occupations,Other construction and related workers,47-4000,Septic tank servicers and sewer pipe cleaners,47-4071,Line item,47,4071
7,47-0000,Construction and extraction occupations,Other construction and related workers,47-4000,Miscellaneous construction and related workers,47-4090,Line item,47,4090
8,47-0000,Construction and extraction occupations,Extraction workers,47-5000,"Derrick, rotary drill, and service unit operat...",47-5010,Summary,47,5010
9,47-0000,Construction and extraction occupations,Extraction workers,47-5000,"Derrick operators, oil and gas",47-5011,Line item,47,5011


In [17]:
file = 'Cleaned_BLS_Employment_Categories.csv'
final_output_df.to_csv('Resources/'+file, index=False)